In [2]:
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.7 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-19.24.4-cp310-cp310-linux_x86_64.whl size=3327203 sha256=5d1d6278223e5d82db51bf321846101570e7f026d9a7739bc15960ff862d3b0d
  Stored in directory: /root/.cache/pip/wheels/08/5e/55/a7db8d57920eb2c2aa41c36dc4c6d3cd12323865de191a7211
Successfully built dlib


In [11]:
!pip install requests
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Raises an exception for HTTP errors
    
    # Save the file
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f'Download complete: {filename}')

# URL to the dlib pre-trained model for facial landmark detection
url = "https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat.bz2"
filename = "shape_predictor_68_face_landmarks.dat.bz2"

download_file(url, filename)

print("Unzipping the file...")

# Unzipping the .bz2 file to get the .dat file
import bz2
with bz2.open(filename, 'rb') as file:
    decompressed_data = file.read()
    with open('shape_predictor_68_face_landmarks.dat', 'wb') as new_file:
        new_file.write(decompressed_data)

print("File is ready to use: shape_predictor_68_face_landmarks.dat")

Download complete: shape_predictor_68_face_landmarks.dat.bz2
Unzipping the file...
File is ready to use: shape_predictor_68_face_landmarks.dat


In [ ]:
import cv2
import dlib
import os
import numpy as np
import librosa

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/kaggle/working/shape_predictor_68_face_landmarks.dat')

# Function to process audio and get MFCC features
def get_mfcc_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

# Function to process video and get facial landmarks for each frame
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_landmarks = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        
        for face in faces:
            shape = predictor(gray, face)
            landmark = np.array([(part.x, part.y) for part in shape.parts()]).flatten()
            frame_landmarks.append(landmark)
    
    cap.release()
    return frame_landmarks

# Function to read data from files and combine features
def read_data_from_files(main_directory_audio, main_directory_video):
    data = []
    
    for class_folder in os.listdir(main_directory_audio):
        class_folder_path_audio = os.path.join(main_directory_audio, class_folder)
        class_folder_path_video = os.path.join(main_directory_video, class_folder)
        
        if os.path.isdir(class_folder_path_audio) and os.path.isdir(class_folder_path_video):
            audio_files = {f[:-4]: f for f in os.listdir(class_folder_path_audio) if f.endswith('.wav')}
            video_files = {f[:-4]: f for f in os.listdir(class_folder_path_video) if f.endswith('.mp4')}
            common_files = set(audio_files.keys()).intersection(video_files.keys())
            
            for base_name in common_files:
                filepath_audio = os.path.join(class_folder_path_audio, audio_files[base_name])
                filepath_video = os.path.join(class_folder_path_video, video_files[base_name])
                print(base_name)
                
                mfcc_features = get_mfcc_features(filepath_audio)
                
                frame_landmarks = process_video(filepath_video)
                
                for landmark in frame_landmarks:
                    if landmark.size > 0:
                        combined_features = np.concatenate([mfcc_features, landmark])
                        data.append((combined_features, class_folder))
                    else:
                        print(f"No landmarks found for frame in video {base_name}, skipping.")
    
    return data

# Example usage
data_directory_audio = '/kaggle/input/real-life-deception-audio/Audio'
data_directory_video = '/kaggle/input/real-life-deception-detection/Real-life_Deception_Detection_2016/Clips'
data = read_data_from_files(data_directory_audio, data_directory_video)

# Separate features and labels
features = np.array([d[0] for d in data])
labels = np.array([d[1] for d in data])

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)


trial_lie_050
trial_lie_032
trial_lie_031
trial_lie_039
trial_lie_059
trial_lie_030
trial_lie_019
trial_lie_043
trial_lie_047
trial_lie_020
trial_lie_007
trial_lie_021
trial_lie_027
trial_lie_002
trial_lie_028
trial_lie_026
trial_lie_058
trial_lie_041
trial_lie_034
trial_lie_054
trial_lie_057
trial_lie_042
trial_lie_009
trial_lie_035
trial_lie_040
trial_lie_033
trial_lie_036
trial_lie_012
trial_lie_015
trial_lie_003
trial_lie_022
trial_lie_014
trial_lie_053
trial_lie_016
trial_lie_049
trial_lie_023
trial_lie_029
trial_lie_025
trial_lie_008
trial_lie_038
trial_lie_044
trial_lie_045
trial_lie_017
trial_lie_024
trial_lie_006
trial_lie_048
trial_lie_061
trial_lie_055
trial_lie_005
trial_lie_004
trial_lie_051
trial_lie_001
trial_lie_052
trial_lie_060
trial_lie_011


In [ ]:
import pickle

# Print the shapes of the features and labels
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

# Save features and labels to a pkl file
with open('V+A.pkl', 'wb') as file:
    pickle.dump((features, labels), file)

print("Data saved to data.pkl")

In [1]:
import pickle

# Load features and labels from the pkl file
with open('V+A.pkl', 'rb') as file:
    features, labels = pickle.load(file)

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

Features shape: (82575, 149)
Labels shape: (82575,)


In [2]:
import pandas as pd

features_df = pd.DataFrame(features)
features_df['label'] = labels

print(features_df)

                0           1          2          3          4          5  \
0     -439.731171  149.665756  14.460912  31.690729   9.315437   8.397162   
1     -439.731171  149.665756  14.460912  31.690729   9.315437   8.397162   
2     -439.731171  149.665756  14.460912  31.690729   9.315437   8.397162   
3     -664.282593  177.635559  -0.003443  11.574381  36.157589  -2.649074   
4     -664.282593  177.635559  -0.003443  11.574381  36.157589  -2.649074   
...           ...         ...        ...        ...        ...        ...   
82570 -409.120544  157.019455 -55.524513  54.097355  15.841450  15.236379   
82571 -409.120544  157.019455 -55.524513  54.097355  15.841450  15.236379   
82572 -409.120544  157.019455 -55.524513  54.097355  15.841450  15.236379   
82573 -409.120544  157.019455 -55.524513  54.097355  15.841450  15.236379   
82574 -409.120544  157.019455 -55.524513  54.097355  15.841450  15.236379   

              6          7         8          9  ...    140    141    142  

In [3]:
from sklearn.model_selection import train_test_split
# Split into training, validation, and test sets
train_df, test_df = train_test_split(features_df, test_size=0.2, random_state=42)
# 0.25 x 0.8 = 0.2
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

In [4]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(49545, 150)
(16515, 150)
(16515, 150)


In [5]:
def separate_features_and_labels(df):
    X = df.drop(columns=['label']).values  # Drop the label column to get features
    y = df['label'].apply(lambda x: 1 if x == 'Truthful' else 0).values  # Convert labels to binary
    return X, y

X_train, y_train = separate_features_and_labels(train_df)
X_val, y_val = separate_features_and_labels(val_df)
X_test, y_test = separate_features_and_labels(test_df)

In [6]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = (X_train.shape[1], X_train.shape[2])  # (timesteps, features)
model = models.Sequential([
    layers.LSTM(1024, input_shape=input_shape, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(512, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(256, return_sequences=False),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 1024)        │     4,808,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 512)         │     3,147,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,744,193 (33.36 MB)

 Trainable params: 8,744,193 (33.36 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
batch_size = 128
epochs = 20

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.7661 - loss: 0.4934 - val_accuracy: 0.8728 - val_loss: 0.2745
Epoch 2/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8337 - loss: 0.3522 - val_accuracy: 0.8372 - val_loss: 0.2801
Epoch 3/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8443 - loss: 0.3291 - val_accuracy: 0.8622 - val_loss: 0.2674
Epoch 4/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8530 - loss: 0.3132 - val_accuracy: 0.8250 - val_loss: 0.3333
Epoch 5/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8517 - loss: 0.3160 - val_accuracy: 0.9160 - val_loss: 0.1993
Epoch 6/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8625 - loss: 0.2852 - val_accuracy: 0.8876 - val_loss: 0.2107
Epoch 7/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8626 - loss: 0.2830 - val_accuracy: 0.8286 - val_loss: 0.3026
Epoch 8/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8528 - loss: 0.3108 - val_accu

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.layers import LSTM, Bidirectional


input_shape = (X_train.shape[1], X_train.shape[2])
model = models.Sequential([
    layers.Bidirectional(LSTM(1024, return_sequences=True), input_shape=input_shape),
    layers.Dropout(0.2),
    layers.Bidirectional(LSTM(512, return_sequences=True)),
    layers.Dropout(0.2),
    layers.Bidirectional(LSTM(256, return_sequences=False)),
    layers.Dense(1, activation='sigmoid'),
                         ])

learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1, 2048)        │     9,617,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 2048)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 1, 1024)        │    10,489,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 512)            │     2,623,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,731,265 (86.71 MB)

 Trainable params: 22,731,265 (86.71 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
batch_size = 128
epochs = 20

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.8023 - loss: 0.4320 - val_accuracy: 0.8538 - val_loss: 0.3301
Epoch 2/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.8392 - loss: 0.3419 - val_accuracy: 0.8765 - val_loss: 0.2604
Epoch 3/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8545 - loss: 0.3076 - val_accuracy: 0.8837 - val_loss: 0.2242
Epoch 4/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8589 - loss: 0.2864 - val_accuracy: 0.8819 - val_loss: 0.2170
Epoch 5/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8537 - loss: 0.2993 - val_accuracy: 0.8872 - val_loss: 0.2195
Epoch 6/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8655 - loss: 0.2730 - val_accuracy: 0.8607 - val_loss: 0.2522
Epoch 7/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8697 - loss: 0.2620 - val_accuracy: 0.8892 - val_loss: 0.1805
Epoch 8/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8675 - loss: 0.2623 - val_ac

In [15]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Concatenate


# Define the LSTM model
timesteps = 1
input_shape = (X_train.shape[1], X_train.shape[2])

input_layer = Input(shape=input_shape)

lstm1 = LSTM(1024, return_sequences=True)(input_layer)
lstm1 = Dropout(0.2)(lstm1)
lstm1 = LSTM(1024, return_sequences=True)(lstm1)
lstm1 = Dropout(0.2)(lstm1)
lstm2 = LSTM(512, return_sequences=True)(lstm1)
lstm2 = Dropout(0.2)(lstm2)

lstm3 = LSTM(1024, return_sequences=True)(lstm2)
lstm3 = Dropout(0.2)(lstm3)
lstm4 = LSTM(1024, return_sequences=True)(lstm3)
lstm4 = Dropout(0.2)(lstm4)
lstm5 = LSTM(512, return_sequences=True)(lstm4)
lstm5 = Dropout(0.2)(lstm5)

xor_layer = Dense(512, activation='relu')(lstm5)

concatenated = Concatenate()([lstm2, xor_layer])

lstm6 = LSTM(1024, return_sequences=True)(concatenated)
lstm6 = Dropout(0.2)(lstm6)
lstm6 = LSTM(512, return_sequences=True)(lstm6)
lstm6 = Dropout(0.2)(lstm6)
lstm7 = LSTM(256, return_sequences=True)(lstm6)
lstm7 = Dropout(0.2)(lstm7)

output_layer = Dense(1, activation='sigmoid')(lstm7[:, -1, :])

model = Model(inputs=input_layer, outputs=output_layer)

initial_learning_rate = 0.00001
opt = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 1, 149)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_24 (LSTM)      │ (None, 1, 1024)   │  4,808,704 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 1, 1024)   │          0 │ lstm_24[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_25 (LSTM)      │ (None, 1, 1024)   │  8,392,704 │ dropout_23[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_24          │ (None, 1, 1024)   │          0 │ lstm_25[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_26 (LSTM)      │ (None, 1, 512)    │  3,147,776 │ dropout_24[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 1, 512)    │          0 │ lstm_26[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_27 (LSTM)      │ (None, 1, 1024)   │  6,295,552 │ dropout_25[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_26          │ (None, 1, 1024)   │          0 │ lstm_27[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_28 (LSTM)      │ (None, 1, 1024)   │  8,392,704 │ dropout_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_27          │ (None, 1, 1024)   │          0 │ lstm_28[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_29 (LSTM)      │ (None, 1, 512)    │  3,147,776 │ dropout_27[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 1, 512)    │          0 │ lstm_29[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1, 512)    │    262,656 │ dropout_28[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 1, 1024)   │          0 │ dropout_25[0][0], │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_30 (LSTM)      │ (None, 1, 1024)   │  8,392,704 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 1, 1024)   │          0 │ lstm_30[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_31 (LSTM)      │ (None, 1, 512)    │  3,147,776 │ dropout_29[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 1, 512)    │          0 │ lstm_31[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_32 (LSTM)      │ (None, 1, 256)    │    787,456 │ dropout_30[0][0]

 Total params: 46,776,065 (178.44 MB)

 Trainable params: 46,776,065 (178.44 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
batch_size = 128
epochs = 20

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - accuracy: 0.5624 - loss: 0.6789 - val_accuracy: 0.8426 - val_loss: 0.3661
Epoch 2/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.8682 - loss: 0.3101 - val_accuracy: 0.9339 - val_loss: 0.1683
Epoch 3/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9071 - loss: 0.2120 - val_accuracy: 0.9398 - val_loss: 0.1395
Epoch 4/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9168 - loss: 0.1807 - val_accuracy: 0.9304 - val_loss: 0.1321
Epoch 5/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9251 - loss: 0.1585 - val_accuracy: 0.9470 - val_loss: 0.1074
Epoch 6/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9256 - loss: 0.1494 - val_accuracy: 0.9361 - val_loss: 0.1039
Epoch 7/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9312 - loss: 0.1432 - val_accuracy: 0.9488 - val_loss: 0.1032
Epoch 8/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9337 - loss: 0.1337 - 